In [1]:
#Setting up the data for the analysis

#Create the connection
conn = create_connection('NYU.db')

#Getting the data into pandas dataframe
posts = pd.read_sql("SELECT * FROM post", conn)
users = pd.read_sql("SELECT * FROM user", conn)
comments = pd.read_sql("SELECT * FROM comment", conn)
subreddits = pd.read_sql("SELECT * FROM subreddit", conn)

#Closing connection
conn.close()

NameError: name 'create_connection' is not defined

In [ ]:
#Removing bots
bots = subreddits[subreddits['name'].str.endswith('bot')]
bot_names = set(bots['name'].tolist())

for name in bot_names:
    users = users[users['name'] != name]
    posts = posts[posts['name'] != name]
    subreddits = subreddits[subreddits['name'] != name]
    comments = comments[comments['name'] != name]

In [ ]:
###### QUESTIONS TO INVESTIGATE ######

'''
1, First post
2, Post with most upvotes/downvotes
3, Comments with most upvotes/downvotes
4, Does NYU subreddit fills with application questions?
5, How often does users posts/comments in NYU subreddit
6, where else do users posts -> most popular subreddits
7, Type of students in NYU subreddit
8, money related posts questions
9, Most common word (using TFDIF), try to remove filler
10, Does NYU subreddit represent NYU in general
11, post activity over the year
12, Frequency of posts/comments in the subreddit
'''